<H2>This notebook contains the code used to analyze the problem: <br>
"Where to open a gym in Bari?"</H2>

In [102]:
# The code was removed by Watson Studio for sharing.

<b>--> Import required packages</b>

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    branca-0.3.0               |             py_0          24 KB  conda-forge
    ------------------------------------------------------------
                      

In [123]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import requests # library to handle requests

import pandas as pd # library for data analsysis
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
import numpy as np # library to handle data in a vectorized manner
import folium # plotting library

from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

<b>--> Use geopy library to get the latitude and longitude values of Bari City</b>

In [4]:
address = 'Bari'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bari are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bari are 41.1257843, 16.8620293.


<b>--> Show position of Bari on the map</b>

In [7]:
map_bari = folium.Map(location=[latitude, longitude], zoom_start=10)
label = 'Bari'
folium.CircleMarker(
    [latitude, longitude],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#blue',
    fill_opacity=0.7,
    parse_html=False).add_to(map_bari) 
    
map_bari

<H1>Collect required data</H1>

<b>--> Set credentials to connect to FourSquare</b>

In [5]:
CLIENT_ID = 'QXJTHOHPJNXWTSIL5L0IGRHOXNPIDHZ2UGIM522UOAHEQDIN' # your Foursquare ID
CLIENT_SECRET = 'VYEQ5FKC5SENLDM1FUABONMDLYJX5WZUEBQGFUX3UMXUZVW1' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QXJTHOHPJNXWTSIL5L0IGRHOXNPIDHZ2UGIM522UOAHEQDIN
CLIENT_SECRET:VYEQ5FKC5SENLDM1FUABONMDLYJX5WZUEBQGFUX3UMXUZVW1


<b>--> Run a query in FoursQuare to extract all Gyms in Bari. I'll use 3 search terms in Italian and English (it is possible that some are tagged in Italian).</b>

In [8]:
radius = 5000

In [9]:
search_query = 'Palestra'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bd837694c1f67197b3d67d6'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'primary': True,
      'shortName': 'Gym'}],
    'hasPerk': False,
    'id': '4f8d91a2e4b05b22fddaf232',
    'location': {'address': 'Via Niccolò Putignani, 134, 70122 Bari',
     'cc': 'IT',
     'city': 'Bari',
     'country': 'Italia',
     'distance': 465,
     'formattedAddress': ['Via Niccolò Putignani, 134, 70122 Bari',
      '70122 Bari Puglia',
      'Italia'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.12317815536922,
       'lng': 16.866367745278342}],
     'lat': 41.12317815536922,
     'lng': 16.866367745278342,
     'postalCode': '70122',
     'state': 'Puglia'},
    'name': 'Palestra Beauty Line',
    'referralId': 'v-1540896617'},
   {'categories': [],
 

<b>--> Save in the dataframe df_gyms the list of all Gyms in Bari.<br>
       I will append results of 3 different queries to the same dataframe.</b>

In [10]:
venues = results['response']['venues']
df_gyms = json_normalize(venues)
df_gyms.shape

(13, 17)

In [11]:
df_gyms

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'shortName': 'Gym', 'icon': {'prefix': 'http...",False,4f8d91a2e4b05b22fddaf232,"Via Niccolò Putignani, 134, 70122 Bari",IT,Bari,Italia,NaN,465,"[Via Niccolò Putignani, 134, 70122 Bari, 70122...","[{'label': 'display', 'lng': 16.86636774527834...",41.123178,16.866368,70122,Puglia,Palestra Beauty Line,v-1540896617
1,[],False,4f146b62e4b0aabf1ab5d193,NaN,IT,NaN,Italia,NaN,1825,[Italia],"[{'label': 'display', 'lng': 16.845952, 'lat':...",41.136836,16.845952,NaN,NaN,Palestra,v-1540896617
2,"[{'shortName': 'Education', 'icon': {'prefix':...",False,4f12ab2ee4b0960d000c7fd4,Via Pagano,IT,NaN,Italia,NaN,1314,"[Via Pagano, Italia]","[{'label': 'display', 'lng': 16.84797437405111...",41.120552,16.847974,NaN,NaN,Palestra S.M.S Carlo Levi,v-1540896617
3,"[{'shortName': 'Gym / Fitness', 'icon': {'pref...",False,4dbf171181545e1cc7c6917c,"via Michelangelo signorile, 55",IT,Bari,Italia,NaN,1587,"[via Michelangelo signorile, 55, 70124 Bari Pu...","[{'label': 'display', 'lng': 16.878119, 'lat':...",41.118272,16.878119,70124,Puglia,Palestra Mondo Fitness,v-1540896617
4,"[{'shortName': 'Gym', 'icon': {'prefix': 'http...",False,4fa0fbb8e4b03ac4dafc515a,NaN,IT,NaN,Italia,NaN,2798,[Italia],"[{'label': 'display', 'lng': 16.854608, 'lat':...",41.101277,16.854608,NaN,NaN,Palestra,v-1540896617
5,"[{'shortName': 'Gym', 'icon': {'prefix': 'http...",False,4fbaaf30e4b01e9233e95ce3,Via luigi ricchioni 34,IT,Bari,Italia,NaN,1129,"[Via luigi ricchioni 34, 70124 Bari Puglia, It...","[{'label': 'display', 'lng': 16.8644, 'lat': 4...",41.115800,16.864400,70124,Puglia,Lei Fit Palestra Per Donne,v-1540896617
6,"[{'shortName': 'Gym', 'icon': {'prefix': 'http...",False,4f4d1bd1e4b05ebee480b3e4,NaN,IT,NaN,Italia,NaN,2141,[Italia],"[{'label': 'display', 'lng': 16.86863820804451...",41.107207,16.868638,NaN,NaN,Palestra Eagles Volley,v-1540896617
7,"[{'shortName': 'School', 'icon': {'prefix': 'h...",False,4efe1bb7775bec6b4267673c,Via Sabotino 1,IT,NaN,Italia,NaN,2155,"[Via Sabotino 1, Italia]","[{'label': 'display', 'lng': 16.86999819149174...",41.107377,16.869998,NaN,NaN,Palestra Ist. Margherita,v-1540896617
8,"[{'shortName': 'Gym', 'icon': {'prefix': 'http...",False,4f4e75b1e4b00b42c56b9143,NaN,IT,NaN,Italia,NaN,1251,[Italia],"[{'label': 'display', 'lng': 16.86743382269536...",41.115307,16.867434,NaN,NaN,gymtonic palestra,v-1540896617
9,"[{'shortName': 'Gym', 'icon': {'prefix': 'http...",False,54062d9e498ed1980149651f,"Via A Omodeo, 45",IT,Bari,Italia,Via Salvemini,2824,"[Via A Omodeo, 45 (Via Salvemini), 70125 Bari ...","[{'label': 'display', 'lng': 16.88255136441807...",41.105661,16.882551,70125,Puglia,Palestra McFIT,v-1540896617


In [12]:
search_query = 'Gym'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bd839bfdd5797073dd98b0d'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d175941735',
      'name': 'Gym / Fitness Center',
      'pluralName': 'Gyms or Fitness Centers',
      'primary': True,
      'shortName': 'Gym / Fitness'}],
    'hasPerk': False,
    'id': '4d49adf05241236ac37f7804',
    'location': {'address': 'Corso Italia, 112',
     'cc': 'IT',
     'city': 'Bari',
     'country': 'Italia',
     'distance': 861,
     'formattedAddress': ['Corso Italia, 112', 'Bari Puglia', 'Italia'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.11833363040973,
       'lng': 16.864823382512572}],
     'lat': 41.11833363040973,
     'lng': 16.864823382512572,
     'state': 'Puglia'},
    'name': "Wellness'N Gym",
    'referralId': 'v-1540897215'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/catego

In [15]:
venues = results['response']['venues']
df_gyms_2 = json_normalize(venues)
df_gyms=df_gyms.append(df_gyms_2)
df_gyms.shape

(20, 17)

In [16]:
search_query = 'Fitness'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bd83cc3db04f50f801759a7'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/pool_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d105941735',
      'name': 'Gym Pool',
      'pluralName': 'Gym Pools',
      'primary': True,
      'shortName': 'Pool'}],
    'hasPerk': False,
    'id': '528bcb1611d25e023bb55eda',
    'location': {'address': 'Via dante 107',
     'cc': 'IT',
     'city': 'Bari',
     'country': 'Italia',
     'distance': 644,
     'formattedAddress': ['Via dante 107', '70122 Bari Puglia', 'Italia'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.121942428872,
       'lng': 16.8677757214755}],
     'lat': 41.121942428872,
     'lng': 16.8677757214755,
     'postalCode': '70122',
     'state': 'Puglia'},
    'name': 'Wakeup fitness benessere estetica',
    'referralId': 'v-1540897987'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/i

In [17]:
venues = results['response']['venues']
df_gyms_3 = json_normalize(venues)
df_gyms=df_gyms.append(df_gyms_3)
df_gyms.shape

(28, 17)

<H1>Data Preparation</H1>

<b>--> Clean dataframe and select important info</b>

In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_gyms.columns if col.startswith('location.')] + ['id']
df_gyms_filtered = df_gyms.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_gyms_filtered['categories'] = df_gyms_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_gyms_filtered.columns = [column.split('.')[-1] for column in df_gyms_filtered.columns]

df_gyms_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Palestra Beauty Line,Gym,"Via Niccolò Putignani, 134, 70122 Bari",IT,Bari,Italia,NaN,465,"[Via Niccolò Putignani, 134, 70122 Bari, 70122...","[{'label': 'display', 'lng': 16.86636774527834...",41.123178,16.866368,70122,Puglia,4f8d91a2e4b05b22fddaf232
1,Palestra,None,NaN,IT,NaN,Italia,NaN,1825,[Italia],"[{'label': 'display', 'lng': 16.845952, 'lat':...",41.136836,16.845952,NaN,NaN,4f146b62e4b0aabf1ab5d193
2,Palestra S.M.S Carlo Levi,General College & University,Via Pagano,IT,NaN,Italia,NaN,1314,"[Via Pagano, Italia]","[{'label': 'display', 'lng': 16.84797437405111...",41.120552,16.847974,NaN,NaN,4f12ab2ee4b0960d000c7fd4
3,Palestra Mondo Fitness,Gym / Fitness Center,"via Michelangelo signorile, 55",IT,Bari,Italia,NaN,1587,"[via Michelangelo signorile, 55, 70124 Bari Pu...","[{'label': 'display', 'lng': 16.878119, 'lat':...",41.118272,16.878119,70124,Puglia,4dbf171181545e1cc7c6917c
4,Palestra,Gym,NaN,IT,NaN,Italia,NaN,2798,[Italia],"[{'label': 'display', 'lng': 16.854608, 'lat':...",41.101277,16.854608,NaN,NaN,4fa0fbb8e4b03ac4dafc515a
5,Lei Fit Palestra Per Donne,Gym,Via luigi ricchioni 34,IT,Bari,Italia,NaN,1129,"[Via luigi ricchioni 34, 70124 Bari Puglia, It...","[{'label': 'display', 'lng': 16.8644, 'lat': 4...",41.115800,16.864400,70124,Puglia,4fbaaf30e4b01e9233e95ce3
6,Palestra Eagles Volley,Gym,NaN,IT,NaN,Italia,NaN,2141,[Italia],"[{'label': 'display', 'lng': 16.86863820804451...",41.107207,16.868638,NaN,NaN,4f4d1bd1e4b05ebee480b3e4
7,Palestra Ist. Margherita,School,Via Sabotino 1,IT,NaN,Italia,NaN,2155,"[Via Sabotino 1, Italia]","[{'label': 'display', 'lng': 16.86999819149174...",41.107377,16.869998,NaN,NaN,4efe1bb7775bec6b4267673c
8,gymtonic palestra,Gym,NaN,IT,NaN,Italia,NaN,1251,[Italia],"[{'label': 'display', 'lng': 16.86743382269536...",41.115307,16.867434,NaN,NaN,4f4e75b1e4b00b42c56b9143
9,Palestra McFIT,Gym,"Via A Omodeo, 45",IT,Bari,Italia,Via Salvemini,2824,"[Via A Omodeo, 45 (Via Salvemini), 70125 Bari ...","[{'label': 'display', 'lng': 16.88255136441807...",41.105661,16.882551,70125,Puglia,54062d9e498ed1980149651f


In [19]:
df_gyms_filtered.shape

(28, 15)

In [24]:
df_gyms_filtered.drop(['labeledLatLngs', 'formattedAddress'], axis=1, inplace=True)
df_gyms_filtered

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state,id
0,Palestra Beauty Line,Gym,"Via Niccolò Putignani, 134, 70122 Bari",IT,Bari,Italia,NaN,465,41.123178,16.866368,70122,Puglia,4f8d91a2e4b05b22fddaf232
1,Palestra,None,NaN,IT,NaN,Italia,NaN,1825,41.136836,16.845952,NaN,NaN,4f146b62e4b0aabf1ab5d193
2,Palestra S.M.S Carlo Levi,General College & University,Via Pagano,IT,NaN,Italia,NaN,1314,41.120552,16.847974,NaN,NaN,4f12ab2ee4b0960d000c7fd4
3,Palestra Mondo Fitness,Gym / Fitness Center,"via Michelangelo signorile, 55",IT,Bari,Italia,NaN,1587,41.118272,16.878119,70124,Puglia,4dbf171181545e1cc7c6917c
4,Palestra,Gym,NaN,IT,NaN,Italia,NaN,2798,41.101277,16.854608,NaN,NaN,4fa0fbb8e4b03ac4dafc515a
5,Lei Fit Palestra Per Donne,Gym,Via luigi ricchioni 34,IT,Bari,Italia,NaN,1129,41.115800,16.864400,70124,Puglia,4fbaaf30e4b01e9233e95ce3
6,Palestra Eagles Volley,Gym,NaN,IT,NaN,Italia,NaN,2141,41.107207,16.868638,NaN,NaN,4f4d1bd1e4b05ebee480b3e4
7,Palestra Ist. Margherita,School,Via Sabotino 1,IT,NaN,Italia,NaN,2155,41.107377,16.869998,NaN,NaN,4efe1bb7775bec6b4267673c
8,gymtonic palestra,Gym,NaN,IT,NaN,Italia,NaN,1251,41.115307,16.867434,NaN,NaN,4f4e75b1e4b00b42c56b9143
9,Palestra McFIT,Gym,"Via A Omodeo, 45",IT,Bari,Italia,Via Salvemini,2824,41.105661,16.882551,70125,Puglia,54062d9e498ed1980149651f


<b>--> Remove duplicate Gyms</b>

In [28]:
df_gyms_filtered.drop_duplicates(keep='first', inplace=True)
df_gyms_filtered

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state,id
0,Palestra Beauty Line,Gym,"Via Niccolò Putignani, 134, 70122 Bari",IT,Bari,Italia,NaN,465,41.123178,16.866368,70122,Puglia,4f8d91a2e4b05b22fddaf232
1,Palestra,None,NaN,IT,NaN,Italia,NaN,1825,41.136836,16.845952,NaN,NaN,4f146b62e4b0aabf1ab5d193
2,Palestra S.M.S Carlo Levi,General College & University,Via Pagano,IT,NaN,Italia,NaN,1314,41.120552,16.847974,NaN,NaN,4f12ab2ee4b0960d000c7fd4
3,Palestra Mondo Fitness,Gym / Fitness Center,"via Michelangelo signorile, 55",IT,Bari,Italia,NaN,1587,41.118272,16.878119,70124,Puglia,4dbf171181545e1cc7c6917c
4,Palestra,Gym,NaN,IT,NaN,Italia,NaN,2798,41.101277,16.854608,NaN,NaN,4fa0fbb8e4b03ac4dafc515a
5,Lei Fit Palestra Per Donne,Gym,Via luigi ricchioni 34,IT,Bari,Italia,NaN,1129,41.115800,16.864400,70124,Puglia,4fbaaf30e4b01e9233e95ce3
6,Palestra Eagles Volley,Gym,NaN,IT,NaN,Italia,NaN,2141,41.107207,16.868638,NaN,NaN,4f4d1bd1e4b05ebee480b3e4
7,Palestra Ist. Margherita,School,Via Sabotino 1,IT,NaN,Italia,NaN,2155,41.107377,16.869998,NaN,NaN,4efe1bb7775bec6b4267673c
8,gymtonic palestra,Gym,NaN,IT,NaN,Italia,NaN,1251,41.115307,16.867434,NaN,NaN,4f4e75b1e4b00b42c56b9143
9,Palestra McFIT,Gym,"Via A Omodeo, 45",IT,Bari,Italia,Via Salvemini,2824,41.105661,16.882551,70125,Puglia,54062d9e498ed1980149651f


<b>--> Reset index</b>

In [30]:
df_gyms_filtered.reset_index(inplace=True, drop=True)
df_gyms_filtered

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state,id
0,Palestra Beauty Line,Gym,"Via Niccolò Putignani, 134, 70122 Bari",IT,Bari,Italia,NaN,465,41.123178,16.866368,70122,Puglia,4f8d91a2e4b05b22fddaf232
1,Palestra,None,NaN,IT,NaN,Italia,NaN,1825,41.136836,16.845952,NaN,NaN,4f146b62e4b0aabf1ab5d193
2,Palestra S.M.S Carlo Levi,General College & University,Via Pagano,IT,NaN,Italia,NaN,1314,41.120552,16.847974,NaN,NaN,4f12ab2ee4b0960d000c7fd4
3,Palestra Mondo Fitness,Gym / Fitness Center,"via Michelangelo signorile, 55",IT,Bari,Italia,NaN,1587,41.118272,16.878119,70124,Puglia,4dbf171181545e1cc7c6917c
4,Palestra,Gym,NaN,IT,NaN,Italia,NaN,2798,41.101277,16.854608,NaN,NaN,4fa0fbb8e4b03ac4dafc515a
5,Lei Fit Palestra Per Donne,Gym,Via luigi ricchioni 34,IT,Bari,Italia,NaN,1129,41.115800,16.864400,70124,Puglia,4fbaaf30e4b01e9233e95ce3
6,Palestra Eagles Volley,Gym,NaN,IT,NaN,Italia,NaN,2141,41.107207,16.868638,NaN,NaN,4f4d1bd1e4b05ebee480b3e4
7,Palestra Ist. Margherita,School,Via Sabotino 1,IT,NaN,Italia,NaN,2155,41.107377,16.869998,NaN,NaN,4efe1bb7775bec6b4267673c
8,gymtonic palestra,Gym,NaN,IT,NaN,Italia,NaN,1251,41.115307,16.867434,NaN,NaN,4f4e75b1e4b00b42c56b9143
9,Palestra McFIT,Gym,"Via A Omodeo, 45",IT,Bari,Italia,Via Salvemini,2824,41.105661,16.882551,70125,Puglia,54062d9e498ed1980149651f


<b>--> Let's get the venues that are around each gym within a radius of 500 meters</b>

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Gym Name', 
                  'Gym Latitude', 
                  'Gym Longitude', 
                  'Venue',  
                  'Venue id',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
venues_around_gyms = getNearbyVenues(names=df_gyms_filtered['name'],
                                   latitudes=df_gyms_filtered['lat'],
                                   longitudes=df_gyms_filtered['lng']
                                  )
venues_around_gyms.shape

Palestra Beauty Line
Palestra
Palestra S.M.S Carlo Levi
Palestra Mondo Fitness
Palestra
Lei Fit Palestra Per Donne
Palestra Eagles Volley
Palestra Ist. Margherita
gymtonic palestra
Palestra McFIT
Palestra New Wave
Palestra mea
My Gym
Wellness'N Gym
Gym
Splendour Of The Seas Gym
Iron Gym
Empire Gym
Wakeup fitness benessere estetica
Fitness Lab
Millenum Fitness Bari
Eleven Fitness Club
Eleven Health & Fitness
Movida Dance & Fitness Club


(350, 8)

In [34]:
venues_around_gyms

,Gym Name,Gym Latitude,Gym Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
0,Palestra Beauty Line,41.123178,16.866368,Cake Design Cupcake & Bakery,4e666d59aeb77c17bf35baac,41.123950,16.868000,Cupcake Shop
1,Palestra Beauty Line,41.123178,16.866368,Frulez,4efa10597ee5bf79953ac8b0,41.121378,16.870101,Vegetarian / Vegan Restaurant
2,Palestra Beauty Line,41.123178,16.866368,Palazzo Delle Poste,4e81fdd693ad2f07c865f33c,41.121135,16.867258,Arts & Entertainment
3,Palestra Beauty Line,41.123178,16.866368,Bacio di Latte,527e0211498e7fdbdb54719f,41.125099,16.869416,Café
4,Palestra Beauty Line,41.123178,16.866368,Prelibata Gelateria Re Vittorio,51ae49c9498ed4b421a0a138,41.125953,16.869191,Ice Cream Shop
5,Palestra Beauty Line,41.123178,16.866368,Louis Vuitton,4d00aa3e21ea6ea8b608439f,41.121633,16.869611,Boutique
6,Palestra Beauty Line,41.123178,16.866368,La Muraya,4de22a6fc65b7a3e2113960b,41.125854,16.864414,Food
7,Palestra Beauty Line,41.123178,16.866368,Veronero Caffè,4e258251aeb7df1e0f49a46f,41.125316,16.868863,Café
8,Palestra Beauty Line,41.123178,16.866368,Bari - Napoli,4cd7ef75a42b236ae7dafc08,41.125018,16.863795,Pizza Place
9,Palestra Beauty Line,41.123178,16.866368,I Tesori di Sicilia,4cf90a595698a09385bb08fc,41.124262,16.867047,Café


In [104]:
project.save_data(data=venues_around_gyms.to_csv(index=False),file_name='venues_around_gyms.csv',overwrite=True)

{'asset_id': '382d4b8a-ea59-464d-a27d-d22e6ecde8ab',
 'bucket_name': 'courseraexam-donotdelete-pr-hgtoz8jaabxwlk',
 'file_name': 'x.csv',
 'message': 'File x.csv has been written successfully to the associated OS'}

<H1>Exploratory Data Analysis</H1>

<b>--> Identify the Gym having highest number of Venues around</b>

In [57]:
venue_pivot=venues_around_gyms.pivot_table(index=['Gym Name', 'Gym Latitude', 'Gym Longitude'], values='Venue', aggfunc='count')
venue_pivot.sort_values(by=['Venue'], inplace=True, ascending=False)
venue_pivot

,,,Venue
Gym Name,Gym Latitude,Gym Longitude,
Palestra Beauty Line,41.123178,16.866368,30
Wakeup fitness benessere estetica,41.121942,16.867776,30
Wellness'N Gym,41.118334,16.864823,27
Palestra Mondo Fitness,41.118272,16.878119,21
Lei Fit Palestra Per Donne,41.115800,16.864400,20
Palestra,41.101277,16.854608,19
Palestra New Wave,41.103424,16.880884,19
Eleven Health & Fitness,41.099781,16.884221,19
gymtonic palestra,41.115307,16.867434,17


In [65]:
venue_pivot.dtypes

Venue    int64
dtype: object

In [105]:
venue_pivot.shape

(24, 1)

<b>--> Print top 3 to the map to identify their position</b>

In [67]:
for lat, lng, label in zip([41.123178, 41.121942, 41.118334], [16.866368, 16.867776, 16.864823], ['Palestra Beauty Line', 'Wakeup fitness benessere estetica', 'Wellness \'N Gym']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(gyms_top_venues) 
    
gyms_top_venues

<b>--> Analyze Venue Frequency by Venue Categories around each Gym</b>

In [108]:
venue_pivot_cat=venues_around_gyms.pivot_table(index=['Gym Name', 'Gym Latitude', 'Gym Longitude'], columns='Venue Category', values='Venue', aggfunc='count')
venue_pivot_cat 

,,Venue Category,Art Museum,Arts & Entertainment,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,Boat or Ferry,Bookstore,Boutique,...,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Steakhouse,Sushi Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant
Gym Name,Gym Latitude,Gym Longitude,,,,,,,,,,,,,,,,,,,,,
Eleven Fitness Club,41.106880,16.881040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Eleven Health & Fitness,41.099781,16.884221,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
Empire Gym,41.100854,16.867457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fitness Lab,41.113225,16.871750,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Gym,41.112899,16.869554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Iron Gym,41.112778,16.870287,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
Lei Fit Palestra Per Donne,41.115800,16.864400,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
Millenum Fitness Bari,41.119448,16.881772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
Movida Dance & Fitness Club,41.107273,16.903282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
venue_pivot_cat.shape

(24, 70)

<H1>Clustering:</H1><br>

<b>--> Prepare the data for k-means clustering</b>

In [110]:
venue_pivot_cat=pd.DataFrame(venue_pivot_cat.to_records())

In [111]:
venue_pivot_cat.fillna(0, inplace=True)
venue_pivot_cat

,Gym Name,Gym Latitude,Gym Longitude,Art Museum,Arts & Entertainment,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,...,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Steakhouse,Sushi Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant
0,Eleven Fitness Club,41.106880,16.881040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Eleven Health & Fitness,41.099781,16.884221,0.0,0.0,0.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,Empire Gym,41.100854,16.867457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Fitness Lab,41.113225,16.871750,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Gym,41.112899,16.869554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,Iron Gym,41.112778,16.870287,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,Lei Fit Palestra Per Donne,41.115800,16.864400,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,Millenum Fitness Bari,41.119448,16.881772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,Movida Dance & Fitness Club,41.107273,16.903282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,My Gym,41.108081,16.873876,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


<b>--> Create clusters of Gyms based on the frequency of Venues by Venue Categories</b>

In [112]:
# set number of clusters
kclusters = 5

venue_pivot_cat_clustering = venue_pivot_cat.drop(['Gym Name', 'Gym Latitude', 'Gym Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_pivot_cat_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 0, 0, 3, 3, 3, 2, 2, 3, 3, 0, 3, 4, 3, 3, 0, 2, 0, 3, 0, 1, 4, 2,
       2], dtype=int32)

<b>--> Add cluster labels to list of Gyms</b>

In [218]:
#gyms_merged = df_gyms_filtered
gyms_merged=venue_pivot_cat

# add clustering labels
gyms_merged['Cluster Labels'] = kmeans.labels_

gyms_merged

,Gym Name,Gym Latitude,Gym Longitude,Art Museum,Arts & Entertainment,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,...,Shopping Mall,Smoke Shop,Snack Place,Steakhouse,Sushi Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
0,Eleven Fitness Club,41.106880,16.881040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,Eleven Health & Fitness,41.099781,16.884221,0.0,0.0,0.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
2,Empire Gym,41.100854,16.867457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,Fitness Lab,41.113225,16.871750,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3
4,Gym,41.112899,16.869554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3
5,Iron Gym,41.112778,16.870287,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3
6,Lei Fit Palestra Per Donne,41.115800,16.864400,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
7,Millenum Fitness Bari,41.119448,16.881772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
8,Movida Dance & Fitness Club,41.107273,16.903282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
9,My Gym,41.108081,16.873876,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [219]:
cluster_summary=gyms_merged.groupby(['Cluster Labels']).count()
cluster_summary['Gym Name']

Cluster Labels
0     6
1     1
2     5
3    10
4     2
Name: Gym Name, dtype: int64

<b>--> Print Clusters on the map</b>

In [220]:
cluster_gyms_bari = folium.Map(location=[latitude, longitude], zoom_start=10)
# add gym to map
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(gyms_merged['Gym Latitude'], gyms_merged['Gym Longitude'], gyms_merged['Gym Name'], venue_pivot_cat['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_gyms_bari)
    
cluster_gyms_bari

<b>--> Explore each cluster</b>

In [221]:
cluster_0=gyms_merged.loc[gyms_venues['Cluster Labels'] == 0]
cluster_0

,Gym Name,Gym Latitude,Gym Longitude,Art Museum,Arts & Entertainment,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,...,Shopping Mall,Smoke Shop,Snack Place,Steakhouse,Sushi Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
1,Eleven Health & Fitness,41.099781,16.884221,0.0,0.0,0.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
2,Empire Gym,41.100854,16.867457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
10,Palestra,41.101277,16.854608,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
15,Palestra McFIT,41.105661,16.882551,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
17,Palestra New Wave,41.103424,16.880884,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
19,Palestra mea,41.093096,16.863415,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [222]:
cluster_0_filtered=cluster_0.drop(['Gym Latitude','Gym Longitude', 'Cluster Labels'], axis=1)

<b>--> Top Venue Categories in the Cluster</b>

In [223]:
num_top_venues = 5

for hood in cluster_0_filtered['Gym Name']:
    print("----"+hood+"----")
    temp = cluster_0_filtered[cluster_0_filtered['Gym Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Eleven Health & Fitness----
            venue  freq
0     Pizza Place   6.0
1            Café   3.0
2             Bar   2.0
3      Steakhouse   1.0
4  Sandwich Place   1.0


----Empire Gym----
               venue  freq
0        Pizza Place   3.0
1               Café   1.0
2  Electronics Store   1.0
3           Platform   0.0
4               Pier   0.0


----Palestra----
         venue  freq
0  Pizza Place   5.0
1         Café   2.0
2          Pub   2.0
3        Hotel   1.0
4          Bar   1.0


----Palestra McFIT----
            venue  freq
0     Pizza Place   2.0
1             Gym   2.0
2            Café   2.0
3  Breakfast Spot   1.0
4             Bar   1.0


----Palestra New Wave----
         venue  freq
0         Café   4.0
1          Gym   3.0
2  Pizza Place   2.0
3          Bar   2.0
4      Brewery   1.0


----Palestra mea----
                venue  freq
0         Pizza Place   4.0
1                 Bar   2.0
2               Hotel   1.0
3  Italian Restaurant   1.0
4         

In [224]:
cluster_1=gyms_merged.loc[gyms_venues['Cluster Labels'] == 1]
cluster_1

,Gym Name,Gym Latitude,Gym Longitude,Art Museum,Arts & Entertainment,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,...,Shopping Mall,Smoke Shop,Snack Place,Steakhouse,Sushi Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
20,Splendour Of The Seas Gym,41.136918,16.86835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [225]:
cluster_1_filtered=cluster_1.drop(['Gym Latitude','Gym Longitude', 'Cluster Labels'], axis=1)

In [226]:
num_top_venues = 5

for hood in cluster_1_filtered['Gym Name']:
    print("----"+hood+"----")
    temp = cluster_1_filtered[cluster_1_filtered['Gym Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Splendour Of The Seas Gym----
             venue  freq
0    Boat or Ferry   9.0
1           Cruise   1.0
2  Harbor / Marina   1.0
3             Pier   1.0
4     Noodle House   0.0




In [227]:
cluster_2=gyms_merged.loc[gyms_venues['Cluster Labels'] == 2]
cluster_2

,Gym Name,Gym Latitude,Gym Longitude,Art Museum,Arts & Entertainment,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,...,Shopping Mall,Smoke Shop,Snack Place,Steakhouse,Sushi Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
6,Lei Fit Palestra Per Donne,41.115800,16.864400,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
7,Millenum Fitness Bari,41.119448,16.881772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
16,Palestra Mondo Fitness,41.118272,16.878119,0.0,0.0,0.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
22,Wellness'N Gym,41.118334,16.864823,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2
23,gymtonic palestra,41.115307,16.867434,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2


In [228]:
cluster_2_filtered=cluster_2.drop(['Gym Latitude','Gym Longitude', 'Cluster Labels'], axis=1)

In [229]:
num_top_venues = 5

for hood in cluster_2_filtered['Gym Name']:
    print("----"+hood+"----")
    temp = cluster_2_filtered[cluster_2_filtered['Gym Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Lei Fit Palestra Per Donne----
       venue  freq
0      Hotel   3.0
1       Café   3.0
2  Gastropub   2.0
3        Pub   2.0
4     Hostel   1.0


----Millenum Fitness Bari----
           venue  freq
0           Café   3.0
1    Pizza Place   3.0
2            Pub   2.0
3          Hotel   2.0
4  Train Station   1.0


----Palestra Mondo Fitness----
                venue  freq
0                Café   5.0
1         Pizza Place   4.0
2               Hotel   3.0
3              Bakery   2.0
4  Italian Restaurant   2.0


----Wellness'N Gym----
                  venue  freq
0  Fast Food Restaurant   3.0
1                   Pub   3.0
2                 Hotel   3.0
3         Train Station   2.0
4                 Plaza   2.0


----gymtonic palestra----
           venue  freq
0            Pub   3.0
1      Gastropub   2.0
2           Café   2.0
3        Brewery   1.0
4  Movie Theater   1.0




In [230]:
cluster_3=gyms_merged.loc[gyms_venues['Cluster Labels'] == 3]
cluster_3

,Gym Name,Gym Latitude,Gym Longitude,Art Museum,Arts & Entertainment,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,...,Shopping Mall,Smoke Shop,Snack Place,Steakhouse,Sushi Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
0,Eleven Fitness Club,41.106880,16.881040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,Fitness Lab,41.113225,16.871750,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3
4,Gym,41.112899,16.869554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3
5,Iron Gym,41.112778,16.870287,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3
8,Movida Dance & Fitness Club,41.107273,16.903282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
9,My Gym,41.108081,16.873876,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3
11,Palestra,41.136836,16.845952,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
13,Palestra Eagles Volley,41.107207,16.868638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
14,Palestra Ist. Margherita,41.107377,16.869998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
18,Palestra S.M.S Carlo Levi,41.120552,16.847974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3


In [231]:
cluster_3_filtered=cluster_3.drop(['Gym Latitude','Gym Longitude', 'Cluster Labels'], axis=1)

In [232]:
num_top_venues = 5

for hood in cluster_3_filtered['Gym Name']:
    print("----"+hood+"----")
    temp = cluster_3_filtered[cluster_3_filtered['Gym Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Eleven Fitness Club----
                    venue  freq
0                    Café   3.0
1                     Gym   2.0
2  Gluten-free Restaurant   1.0
3                     Pub   1.0
4             Pastry Shop   1.0


----Fitness Lab----
          venue  freq
0    Art Museum   1.0
1    Steakhouse   1.0
2  Dessert Shop   1.0
3           Pub   1.0
4      Sake Bar   1.0


----Gym----
                venue  freq
0                 Pub   2.0
1           Gastropub   1.0
2  Italian Restaurant   1.0
3          Steakhouse   1.0
4                Café   1.0


----Iron Gym----
                   venue  freq
0             Art Museum   1.0
1             Steakhouse   1.0
2      Mobile Phone Shop   1.0
3          Movie Theater   1.0
4  Outdoors & Recreation   1.0


----Movida Dance & Fitness Club----
            venue  freq
0     Pizza Place   1.0
1    Home Service   1.0
2       Pet Store   1.0
3  Discount Store   1.0
4    Noodle House   0.0


----My Gym----
                    venue  freq
0       

In [233]:
cluster_4=gyms_merged.loc[gyms_venues['Cluster Labels'] == 4]
cluster_4

,Gym Name,Gym Latitude,Gym Longitude,Art Museum,Arts & Entertainment,Athletics & Sports,Bakery,Bar,Bed & Breakfast,Beer Garden,...,Shopping Mall,Smoke Shop,Snack Place,Steakhouse,Sushi Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Cluster Labels
12,Palestra Beauty Line,41.123178,16.866368,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,4
21,Wakeup fitness benessere estetica,41.121942,16.867776,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,4


In [234]:
cluster_4_filtered=cluster_4.drop(['Gym Latitude','Gym Longitude', 'Cluster Labels'], axis=1)

In [235]:
num_top_venues = 5

for hood in cluster_4_filtered['Gym Name']:
    print("----"+hood+"----")
    temp = cluster_4_filtered[cluster_4_filtered['Gym Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Palestra Beauty Line----
                           venue  freq
0                           Café   6.0
1                    Pizza Place   3.0
2  Vegetarian / Vegan Restaurant   2.0
3             Italian Restaurant   2.0
4             Seafood Restaurant   2.0


----Wakeup fitness benessere estetica----
                           venue  freq
0                           Café   5.0
1             Italian Restaurant   2.0
2                          Plaza   2.0
3                    Coffee Shop   2.0
4  Vegetarian / Vegan Restaurant   2.0


